In [ ]:
import numpy as np 
from astropy.modeling import models 

from petrofit.fitting import model_to_image

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['image.origin'] = 'lower'
#plt.rcParams.update({'font.size': 17})

SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

#plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

def savefig(filename):
    plt.savefig("plots/"+filename, dpi=70, bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
@models.custom_model
def ThetaSersic2D(x, y, amplitude=1, r_eff=10, r_in=10, r_out=50, n=4, x_0=50, y_0=50, ellip=0.7, theta=0, m=2):
    """Two dimensional Sersic profile function."""
    cls = models.Sersic2D
    if cls._gammaincinv is None:
        from scipy.special import gammaincinv
        cls._gammaincinv = gammaincinv

    bn = cls._gammaincinv(2. * n, 0.5)
    a, b = r_eff, (1 - ellip) * r_eff
    cos_theta, sin_theta = np.cos(theta), np.sin(theta)
    x_maj = (x - x_0) * cos_theta + (y - y_0) * sin_theta
    x_min = -(x - x_0) * sin_theta + (y - y_0) * cos_theta
    z = np.sqrt((x_maj / a) ** 2 + (x_min / b) ** 2)
    
    theta = np.ones_like(z) * theta
    
    theta[np.where(z < r_in)] = (z[np.where(z < r_in)]/r_in) * np.pi * m

    bn = cls._gammaincinv(2. * n, 0.5)
    a, b = r_eff, (1 - ellip) * r_eff
    cos_theta, sin_theta = np.cos(theta), np.sin(theta)
    x_maj = (x - x_0) * cos_theta + (y - y_0) * sin_theta
    x_min = -(x - x_0) * sin_theta + (y - y_0) * cos_theta
    z = np.sqrt((x_maj / a) ** 2 + (x_min / b) ** 2)
    
    
    return amplitude * np.exp(-bn * (z ** (1 / n) - 1))

In [ ]:
@models.custom_model
def LogThetaSersic2D(x, y, amplitude=1, r_eff=10, r_in=10, r_out=50, n=4, x_0=50, y_0=50, ellip=0.7, theta=0, m=2):
    """Two dimensional Sersic profile function."""
    cls = models.Sersic2D
    if cls._gammaincinv is None:
        from scipy.special import gammaincinv
        cls._gammaincinv = gammaincinv

    bn = cls._gammaincinv(2. * n, 0.5)
    a, b = r_eff, (1 - ellip) * r_eff
    cos_theta, sin_theta = np.cos(theta), np.sin(theta)
    x_maj = (x - x_0) * cos_theta + (y - y_0) * sin_theta
    x_min = -(x - x_0) * sin_theta + (y - y_0) * cos_theta
    z = np.sqrt((x_maj / a) ** 2 + (x_min / b) ** 2)
    
    theta = np.ones_like(z) * theta
    
    theta[np.where(z < r_in)] = (z[np.where(z < r_in)]/r_in) * np.pi * m

    bn = cls._gammaincinv(2. * n, 0.5)
    a, b = r_eff, (1 - ellip) * r_eff
    cos_theta, sin_theta = np.cos(theta), np.sin(theta)
    x_maj = (x - x_0) * cos_theta + (y - y_0) * sin_theta
    x_min = -(x - x_0) * sin_theta + (y - y_0) * cos_theta
    z = np.sqrt((x_maj / a) ** 2 + (x_min / b) ** 2)
    
    
    return np.log10(amplitude * np.exp(-bn * (z ** (1 / n) - 1)))

In [ ]:
@models.custom_model
def BarSersic2D(x, y, amplitude=1, r_eff=10, n=1, x_0=50, y_0=50, ellip=1, theta=0, c0=0):
    """Two dimensional Sersic profile function."""
    cls = models.Sersic2D
    if cls._gammaincinv is None:
        try:
            from scipy.special import gammaincinv
            cls._gammaincinv = gammaincinv
        except ValueError:
            raise ImportError('Sersic2D model requires scipy.')

    bn = cls._gammaincinv(2. * n, 0.5)
    a, b = r_eff, (1 - ellip) * r_eff
    cos_theta, sin_theta = np.cos(theta), np.sin(theta)
    x_maj = (x - x_0) * cos_theta + (y - y_0) * sin_theta
    x_min = -(x - x_0) * sin_theta + (y - y_0) * cos_theta
    z = (abs(x_maj / a) ** (c0+2) + abs(x_min / b) ** (c0+2))**(1/(c0+2))

    return amplitude * np.exp(-bn * (z ** (1 / n) - 1))

In [ ]:
fig, ax = plt.subplots(1,3, figsize=[8*3, 8])

plt.sca(ax[0])
plt.imshow(model_to_image(50, 50, 120, ThetaSersic2D(ellip=0.5,)), vmax=0.2, vmin=-0.1)
plt.title("Rotating Sérsic Model")
plt.xlabel("Pixels")
plt.ylabel("Pixels")


plt.sca(ax[1])
im = model_to_image(50, 50, 300, LogThetaSersic2D(ellip=0.3, n=4))
plt.imshow(im+abs(im.min()), vmax=4, vmin=-1)
plt.title("Rotating Log-Sérsic Model")
plt.xlabel("Pixels")
plt.ylabel("Pixels")

plt.sca(ax[2])
plt.imshow(model_to_image(50, 50, 300, BarSersic2D(ellip=0.5, n=4, c0=1.2)), vmax=0.01, vmin=-0.005)
plt.title("Bar Sérsic Model")
plt.xlabel("Pixels")
plt.ylabel("Pixels")

savefig('custom_models.png')
